## Scraping Wikipedia

In [1]:
import requests
from bs4 import BeautifulSoup

def obtener_texto_wikipedia(url):
    # Hacer una solicitud HTTP para obtener el contenido de la página
    respuesta = requests.get(url)

    # Parsear el contenido HTML de la página
    soup = BeautifulSoup(respuesta.text, 'html.parser')

    # Extraer el texto del contenido principal de la página de Wikipedia
    contenido = soup.find('div', {'class': 'mw-parser-output'})
    texto = contenido.get_text(separator='\n')

    return texto

# URL de la página de Wikipedia sobre Inteligencia Artificial
url = 'https://es.wikipedia.org/wiki/Inteligencia_artificial'

# Obtener el texto
texto_wikipedia = obtener_texto_wikipedia(url)

# Imprimir los primeros 500 caracteres para verificar
# print(texto_wikipedia[:500])


## Limpieza de datos

In [2]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def limpiar_texto(texto):
    # Eliminar etiquetas HTML
    texto = re.sub(r'<[^>]+>', '', texto)

    # Eliminar referencias y enlaces
    texto = re.sub(r'\[\d+\]', '', texto)

    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar puntuación y números
    texto = re.sub(r'[^a-záéíóúñ ]', '', texto)

    # Tokenización
    tokens = word_tokenize(texto, language='spanish')

    # Eliminar palabras de parada
    stop_words = set(stopwords.words('spanish'))
    tokens = [palabra for palabra in tokens if palabra not in stop_words]

    return ' '.join(tokens)

# Aplicar la función de limpieza al texto
texto_limpio = limpiar_texto(texto_wikipedia)

print(texto_limpio[:500])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


acuarela alan turing generada mediante inteligencia artificial considerado padre mismacitarequeridala inteligencia artificial ia contexto ciencias computación disciplina conjunto capacidades cognoscitivas intelectuales expresadas sistemas informáticos combinaciones algoritmos cuyo propósito creación máquinas imiten inteligencia humana realizar tareas pueden mejorar conforme recopilen información hizo presente después segunda guerra mundial desarrollo prueba turing mientras locución acuñada infor


## Generar preguntas a partir de los datos

In [3]:
from nltk.tokenize import sent_tokenize

def generar_preguntas_respuestas(texto):
    preguntas_respuestas = []
    oraciones = sent_tokenize(texto)

    for oracion in oraciones:
        # Aquí implementas una lógica para formular una pregunta basada en la oración
        pregunta = "Que es la IA?"
        respuesta = oracion  # En este caso simple, la oración completa es la respuesta

        preguntas_respuestas.append({
            "context": texto,
            "question": pregunta,
            "answer": respuesta
        })

    return preguntas_respuestas

# Ejemplo de uso
conjunto_datos = generar_preguntas_respuestas(texto_limpio)


## Convertir los datos en formato BERT y pasarlo a JSON

In [4]:
import json

def formatear_para_bert(preguntas_respuestas):
    formato_bert = []

    for pr in preguntas_respuestas:
        entrada = {
            'context': pr['context'],
            'qas': [{
                'question': pr['question'],
                'answers': [{
                    'text': pr['answer'],
                    'answer_start': pr['context'].find(pr['answer'])
                }]
            }]
        }
        formato_bert.append(entrada)

    return formato_bert

# Convertir a formato BERT
datos_bert = formatear_para_bert(conjunto_datos)

# Guardar en un archivo JSON
with open('datos_entrenamiento_bert.json', 'w') as f:
    json.dump(datos_bert, f, ensure_ascii=False, indent=4)


## Entrenamiento de datos

In [5]:
from transformers import BertTokenizer, BertForQuestionAnswering
from torch.optim import AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
import json

# Cargar el conjunto de datos
with open('datos_entrenamiento_bert.json', 'r') as f:
    conjunto_datos = json.load(f)

# Cargar el tokenizer y el modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Preparar los datos para BERT
input_ids = []
attention_masks = []
start_positions = []
end_positions = []

for item in conjunto_datos:
    contexto = item['context']
    
    for qa in item['qas']:
        pregunta = qa['question']
        respuesta = qa['answers'][0]['text']
        respuesta_start = qa['answers'][0]['answer_start']

        # Codificar contexto y pregunta
        encoded_dict = tokenizer.encode_plus(
                            contexto,
                            pregunta,
                            max_length=512,
                            pad_to_max_length=True,
                            return_attention_mask=True,
                            truncation=True,
                            return_tensors='pt'
                       )
        
        # Agregar input_ids y attention_masks
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

        # Convertir respuesta_start a token index
        respuesta_span = tokenizer.encode(respuesta, add_special_tokens=False)
        respuesta_start_index = encoded_dict['input_ids'][0].tolist().index(respuesta_span[0])
        respuesta_end_index = respuesta_start_index + len(respuesta_span) - 1

        # Agregar start_positions y end_positions
        start_positions.append(respuesta_start_index)
        end_positions.append(respuesta_end_index)

# Convertir las listas en tensores
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Crear el DataLoader
batch_size = 16
train_data = TensorDataset(input_ids, attention_masks, start_positions, end_positions)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Optimizador
optimizer = AdamW(model.parameters(), lr=2e-5)

c:\Users\Adron\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Adron\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `m

In [6]:
# Entrenamiento
model.train()
for epoch in range(1000):  # Número de épocas
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_start_positions = batch[2]
        b_end_positions = batch[3]

        # Forward pass
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        start_positions=b_start_positions, 
                        end_positions=b_end_positions)

        loss = outputs[0]

        # Backward pass
        loss.backward()

        # Actualizar los parámetros y avanzar
        optimizer.step()
        model.zero_grad()

## Guardar el modelo ya entrenado

In [7]:
# Guardar el modelo
model.save_pretrained('model')

# Cargar el modelo
model = BertForQuestionAnswering.from_pretrained('model')


## Evaluar modelo

In [11]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

## Hacer predicciones

In [13]:
import torch

# Formular una pregunta y su contexto
pregunta = "Que es la IA?"
contexto = "Inteligencia artificial"

# Preparar la entrada para el modelo
inputs = tokenizer.encode_plus(pregunta, contexto, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# Realizar la predicción
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Encontrar la posición de inicio y fin de la respuesta en el contexto
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Convertir las posiciones de respuesta a texto
respuesta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

# Imprimir la respuesta
print("Respuesta:", respuesta)


Respuesta: [CLS] que es la ia ? [SEP] inteligencia


In [14]:
from sklearn.metrics import f1_score

def calcular_f1(respuestas_modelo, respuestas_referencia):
    # Asegúrate de que las respuestas estén en formato de lista y sean binarias (1 para correcto, 0 para incorrecto)
    respuestas_modelo_binarias = [1 if respuesta in respuestas_referencia else 0 for respuesta in respuestas_modelo]
    respuestas_referencia_binarias = [1 for _ in respuestas_referencia]
    
    return f1_score(respuestas_referencia_binarias, respuestas_modelo_binarias)

def respuesta_es_correcta(respuesta_modelo, respuestas_referencia):
    # Considera una respuesta correcta si coincide exactamente con alguna respuesta de referencia
    return 1 if respuesta_modelo in respuestas_referencia else 0


# Respuestas inventadas para probar el F1 Score
respuestas_modelo = []
for i in respuesta:
    respuestas_modelo.append(i)
print(respuestas_modelo)
respuestas_referencia = [
'[', 'S', 'E', 'P', ']', ' ', 'i', 'n', 't', 'e', 'l', 'i', 'g', 'e', 'n'
]

respuestas_modelo_binarias = [respuesta_es_correcta(respuesta, respuestas_referencia) for respuesta in respuestas_modelo]

print(respuestas_modelo_binarias)

f1 = calcular_f1(respuestas_modelo, respuestas_referencia)
print("F1 Score:", f1)


['[', 'C', 'L', 'S', ']', ' ', 'q', 'u', 'e', ' ', 'e', 's', ' ', 'l', 'a', ' ', 'i', 'a', ' ', '?', ' ', '[', 'S', 'E', 'P', ']', ' ', 'i', 'n', 't', 'e', 'l', 'i', 'g', 'e', 'n', 'c', 'i', 'a']
[1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]


ValueError: Found input variables with inconsistent numbers of samples: [15, 39]